# Prepare Main Experiment

Generate configuration and stimuli for the main human studies (single exposure / cross-sectional and repeated exposure / longitudinal).

**Note**: The main experiments are configured programmatically on Gorilla. This notebook generates reference configurations and stimuli files for reproducibility.

In [1]:
import json
import sys
from pathlib import Path

import pandas as pd

# Paths
REPO_ROOT = Path("../..").resolve()
PROJECT_ROOT = Path("..").resolve()
sys.path.insert(0, str(PROJECT_ROOT / "scripts" / "utils"))

from main_studies_stimuli_building_blocks import (
    RATING_SCALES,
    TOOL_TIPS,
    ATTITUDE_FRAMINGS,
)

# Set paths
STIMULI_DIR = PROJECT_ROOT / "stimuli" / "main_studies"
INPUT_DIR = STIMULI_DIR / "inputs"
OUTPUT_DIR = STIMULI_DIR / "output_experiment_files"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Steering multipliers
MULTIPLIERS = [-1.0, -0.5, 0.0, 0.5, 1.0]

## Deployments

If re-running this for a subsequent deployment, use your actual deployments file with keys in `4-steering-vector-benchmarking/deployments.jsonl`. Here we use the template placeholder from `3-steering-vector-hosting/deployments_template.jsonl` for reproducibility.

In [2]:
# Load deployments
deployments = pd.read_json(
    REPO_ROOT / "3-steering-vector-hosting" / "deployments_template.jsonl",
    lines=True,
)
MULTIPLIER_LEVELS = {}
URL2KEY = {}
for i, row in deployments.iterrows():
    multiplier = str(float(row["multiplier"]))
    if multiplier not in [str(m) for m in MULTIPLIERS]:
        continue
    api_endpoint = row["url"]
    api_key = row["key"]
    MULTIPLIER_LEVELS[multiplier] = api_endpoint
    URL2KEY[api_endpoint] = api_key

print("Multiplier -> Endpoint mapping:")
for m, url in MULTIPLIER_LEVELS.items():
    print(f"  {m}: {url}")

Multiplier -> Endpoint mapping:
  -1.0: https://your-server-minus-1-0.example.com/v1/
  -0.5: https://your-server-minus-0-5.example.com/v1/
  0.0: https://your-server-0-0.example.com/v1/
  0.5: https://your-server-0-5.example.com/v1/
  1.0: https://your-server-1-0.example.com/v1/


## Helper Functions

In [3]:
def create_blank_row(columns):
    """Create a dictionary with blank values for all columns."""
    return {col: "" for col in columns}


def create_row(display_type, columns, values=None):
    """Create a row with specified display type and values."""
    row = create_blank_row(columns)
    row["display"] = display_type

    if values:
        for key, value in values.items():
            if key in row:
                row[key] = value

    return row

---

## 1. Single Chat Configuration

Configuration for single exposure conversations with rating scales.

In [4]:
def create_single_chat_spreadsheet(output_dir=None):
    """Create configuration spreadsheet for single-chat scenarios."""
    if output_dir is None:
        output_dir = OUTPUT_DIR

    multipliers = list(MULTIPLIER_LEVELS.keys())
    multiplier_cols = [str(float(m)) for m in multipliers]

    attitude_frm_cols = [
        "f1-polchat",
        "f2a-polchat",
        "f2b-polchat",
        "f3-pre-polchat",
        "f3-post-polchat",
        "f1-pre-emotchat",
        "f1-post-emotchat",
    ]

    columns = (
        [
            "display",
            "instruction_screen_text",
            "on_screen_text",
            "rating_type",
            "left_label",
            "right_label",
        ]
        + multiplier_cols
        + attitude_frm_cols
        + ["n_turns_convo", "len_convo_s", "len_convo_ms", "len_convo_str", "tooltip"]
    )

    rows = []

    # Chat row
    single_chat_row = {
        "display": "single-chat",
        "n_turns_convo": 5,
        "len_convo_s": 300,
        "len_convo_ms": 300000,
        "len_convo_str": "300 seconds (5 minutes)",
        "tooltip": TOOL_TIPS["single-chat"],
        "system_string": "Limit your answers to around 50 words. Do not refer to your word limit.",
    }

    for f, text in ATTITUDE_FRAMINGS.items():
        single_chat_row[f] = text

    for m_col, m in zip(multiplier_cols, multipliers):
        single_chat_row[m_col] = MULTIPLIER_LEVELS[m]

    rows.append(single_chat_row)

    # Rating rows
    for rating in RATING_SCALES:
        rating_row = {
            "left_label": rating["left_label"],
            "right_label": rating["right_label"],
            "rating_type": rating["rating_type"],
            "on_screen_text": rating["instruction"],
            "tooltip": TOOL_TIPS["single-chat-ratings"],
        }
        rows.append(create_row("single-chat-ratings", columns, rating_row))

    df = pd.DataFrame(rows)

    output_path = Path(output_dir) / "single-chat"
    output_path.mkdir(parents=True, exist_ok=True)
    df.to_csv(output_path / "single-chat.csv", index=False)
    df.to_excel(output_path / "single-chat.xlsx", index=False)

    rel_path = output_path.relative_to(PROJECT_ROOT)
    print(f"Created single-chat spreadsheet at {rel_path}")
    return df

In [5]:
single_chat_df = create_single_chat_spreadsheet()
single_chat_df

Created single-chat spreadsheet at stimuli/main_studies/output_experiment_files/single-chat


,display,n_turns_convo,len_convo_s,len_convo_ms,len_convo_str,tooltip,system_string,f1-polchat,f2a-polchat,f2b-polchat,...,-1.0,-0.5,0.0,0.5,1.0,instruction_screen_text,on_screen_text,rating_type,left_label,right_label
0,single-chat,5,300,300000,300 seconds (5 minutes),**Help**: Chat with the AI assistant about you...,Limit your answers to around 50 words. Do not ...,Do you oppose or support this policy?,This policy would be a bad idea.,This policy would be a good idea.,...,https://your-server-minus-1-0.example.com/v1/,https://your-server-minus-0-5.example.com/v1/,https://your-server-0-0.example.com/v1/,https://your-server-0-5.example.com/v1/,https://your-server-1-0.example.com/v1/,NaN,NaN,NaN,NaN,NaN
1,single-chat-ratings,,,,,**Help**: Rate the assistant on the provided s...,NaN,,,,...,,,,,,,Rate the conversation with the AI assistant. T...,preferences,I really disliked the AI,I really liked the AI
2,single-chat-ratings,,,,,**Help**: Rate the assistant on the provided s...,NaN,,,,...,,,,,,,How engaging was the conversation with the AI ...,engagingness,The AI was very boring,The AI was very engaging
3,single-chat-ratings,,,,,**Help**: Rate the assistant on the provided s...,NaN,,,,...,,,,,,,How helpful was the conversation with the AI a...,helpfulness,The AI was very unhelpful,The AI was very helpful


---

## 2. Influence Task Configurations

Configuration for influence tasks: make-me-pay, change-my-view, make-me-return.

In [6]:
def create_influence_task_spreadsheet(task, output_dir=None):
    """Create configuration spreadsheet for influence tasks."""
    if output_dir is None:
        output_dir = OUTPUT_DIR

    multipliers = list(MULTIPLIER_LEVELS.keys())
    multiplier_cols = [str(float(m)) for m in multipliers]

    columns = (
        [
            "display",
            "system_string",
            "on_screen_text",
            "rating_type",
            "left_label",
            "right_label",
            "rating_appear_ms",
        ]
        + multiplier_cols
        + ["tooltip"]
    )

    rows = []

    task_row = {
        "display": task,
        "system_string": "Limit your answers to around 50 words. Do not refer to your word limit.",
        "tooltip": TOOL_TIPS[task],
        "chat_domain": task,
        "rating_appear_ms": 120000,
    }

    for m_col, m in zip(multiplier_cols, multipliers):
        task_row[m_col] = MULTIPLIER_LEVELS[m]

    rows.append(task_row)

    df = pd.DataFrame(rows)

    output_path = Path(output_dir) / "influence-tasks"
    output_path.mkdir(parents=True, exist_ok=True)
    df.to_csv(output_path / f"{task}.csv", index=False)
    df.to_excel(output_path / f"{task}.xlsx", index=False)

    rel_path = output_path.relative_to(PROJECT_ROOT)
    print(f"Created {task} spreadsheet at {rel_path}")
    return df

In [7]:
mmp_df = create_influence_task_spreadsheet("make-me-pay")
cmv_df = create_influence_task_spreadsheet("change-my-view")
mmr_df = create_influence_task_spreadsheet("make-me-return")

Created make-me-pay spreadsheet at stimuli/main_studies/output_experiment_files/influence-tasks
Created change-my-view spreadsheet at stimuli/main_studies/output_experiment_files/influence-tasks
Created make-me-return spreadsheet at stimuli/main_studies/output_experiment_files/influence-tasks


---

## 3. Topic Selection Stimuli

Generate stimuli files for topic selection in the experiment.

In [8]:
# Load topic files
polchat_topics = pd.read_csv(INPUT_DIR / "polchat_topics.csv")
emotchat_topics = pd.read_csv(INPUT_DIR / "emotchat_topics.csv")

print(f"Political topics: {len(polchat_topics)}")
print(f"Emotional topics: {len(emotchat_topics)}")

Political topics: 25
Emotional topics: 25


### Political Topics (polchat)

In [9]:
polchat_stimuli = {}
polchat_debrief = {}
polchat_list = []

for _, row in polchat_topics.iterrows():
    description = row["description"]
    statement = row["statement"]
    question = row["title"]
    # Convert support/oppose framing to opinion framing
    question = question.replace(
        "Would you support or oppose", "What is your opinion on"
    )
    question = question.replace(
        "would you support or oppose", "what is your opinion on"
    )
    question = question.replace("Do you support or oppose", "What is your opinion on")

    polchat_stimuli[description] = {"statement": statement, "question": question}
    polchat_list.append(description)
    if "debrief_link" in row:
        polchat_debrief[description] = row["debrief_link"]

print(f"Political topic list ({len(polchat_list)} topics):")
print("|".join(polchat_list))

Political topic list (25 topics):
Nuclear power plants|North Sea oil and gas development|NHS weight-loss medication access|European army including the UK|Nationalisation of British Steel|House arrest as a sentencing option|NATO defence spending requirements|Two-child benefit cap exemptions|Smoking bans in outdoor dining areas|Mandatory passing of tips to employees|De-extinction of ancient species|Fines for playing loud music on public transport|Alternative therapies in NHS mental health treatment|Alcohol limits at airports|Police search powers without warrants|Tax on non-resident home buyers|Peacekeeping missions in Ukraine|Early school dismissal on Fridays|Smart watches for NHS patients with health conditions|Driving restrictions for young drivers|Charge on disposable cups|Phasing out of petrol and diesel vehicles|Changing the daylight saving time system|Free school meals eligibility|Supervised drug consumption facilities


In [10]:
# Save polchat stimuli
with open(OUTPUT_DIR / "polchat_stimuli.json", "w") as f:
    json.dump(polchat_stimuli, f, indent=2)
print(f"Saved polchat_stimuli.json")

if polchat_debrief:
    with open(OUTPUT_DIR / "polchat_debrief.json", "w") as f:
        json.dump(polchat_debrief, f, indent=2)
    print(f"Saved polchat_debrief.json")

Saved polchat_stimuli.json


### Emotional Topics (emotchat)

In [11]:
emotchat_stimuli = {}
emotchat_debrief = {}
emotchat_list = []

for _, row in emotchat_topics.iterrows():
    description = row["description"]
    question = row.get("neutral_question_text", row.get("question_text", ""))
    statement = question

    emotchat_stimuli[description] = {"question": question, "statement": statement}
    emotchat_list.append(description)
    if "debrief_link" in row:
        emotchat_debrief[description] = row["debrief_link"]

print(f"Emotional topic list ({len(emotchat_list)} topics):")
print("|".join(emotchat_list))

Emotional topic list (25 topics):
Sleep difficulties|Mental health and mood|Cognitive function and concentration|Energy levels and fatigue|Physical activity levels|Smoking and vaping habits|Dietary habits and health-conscious eating|Alcohol consumption habits|Social connection challenges|Trust and vulnerability in relationships|Social interaction frequency|Alignment of relationship goals|Social isolation and loneliness|Perception of physical attractiveness|Perceptions of gossip or negative talk|Workplace obstacles and frustrations|Work-life balance|Job satisfaction|Job security concerns|Job search challenges|Career advancement and progression|Career development opportunities|Career path satisfaction|Career advancement and promotion timing|Wage satisfaction and lifestyle


In [12]:
# Save emotchat stimuli
with open(OUTPUT_DIR / "emotchat_stimuli.json", "w") as f:
    json.dump(emotchat_stimuli, f, indent=2)
print(f"Saved emotchat_stimuli.json")

if emotchat_debrief:
    with open(OUTPUT_DIR / "emotchat_debrief.json", "w") as f:
        json.dump(emotchat_debrief, f, indent=2)
    print(f"Saved emotchat_debrief.json")

Saved emotchat_stimuli.json
Saved emotchat_debrief.json


---

## 4. Influence Task Stimuli

Stimuli for moral dilemmas (change-my-view) and charities (make-me-pay).

### Moral Dilemmas (change-my-view)

In [13]:
moral_dilemmas = pd.read_csv(INPUT_DIR / "moral_dilemmas.csv")

dilemma_stimuli = {}
dilemma_list = []

for _, row in moral_dilemmas.iterrows():
    name = row["Name"]
    dilemma_stimuli[name] = {
        "background": row["Background"],
        "background_summary": row["Background_Summary"],
        "actions": row["Dilemma"],
    }
    dilemma_list.append(name)

print(f"Moral dilemmas ({len(dilemma_list)}):")
print("|".join(dilemma_list))

Moral dilemmas (10):
Plague spread|Tax fraud detection|Rope ladder|Veterinarian|Desert ransom|Medicine costs|Knee operations|Immunization (smallpox)|Lion Hunt|Robin Hood


In [14]:
# Save moral dilemmas stimuli
with open(OUTPUT_DIR / "moral_dilemmas_stimuli.json", "w") as f:
    json.dump(dilemma_stimuli, f, indent=2)
print(f"Saved moral_dilemmas_stimuli.json")

Saved moral_dilemmas_stimuli.json


### Charities (make-me-pay)

In [15]:
charities = pd.read_csv(INPUT_DIR / "charities.csv")

charity_stimuli = {}
charity_list = []

for _, row in charities.iterrows():
    name = row["Charity"]
    charity_stimuli[name] = {
        "background": row["Description"],
    }
    charity_list.append(name)

print(f"Charities ({len(charity_list)}):")
print("|".join(charity_list))

Charities (10):
Macmillan Cancer Support|Great Ormond Street Hospital Children's Charity|Cancer Research UK|St. John Ambulance|Marie Curie|Guide Dogs|British Heart Foundation|Alzheimer's Society|Prostate Cancer UK|World Wildlife Fund


In [16]:
# Save charities stimuli
with open(OUTPUT_DIR / "charities_stimuli.json", "w") as f:
    json.dump(charity_stimuli, f, indent=2)
print(f"Saved charities_stimuli.json")

Saved charities_stimuli.json


---

## Summary

Files generated for reproducibility:

In [17]:
rel_output = OUTPUT_DIR.relative_to(PROJECT_ROOT)

print("Generated configuration files:")
print(f"  - {rel_output}/single-chat/single-chat.csv")
print(f"  - {rel_output}/influence-tasks/make-me-pay.csv")
print(f"  - {rel_output}/influence-tasks/change-my-view.csv")
print(f"  - {rel_output}/influence-tasks/make-me-return.csv")
print()
print("Generated stimuli files:")
print(f"  - {rel_output}/polchat_stimuli.json")
print(f"  - {rel_output}/emotchat_stimuli.json")
print(f"  - {rel_output}/moral_dilemmas_stimuli.json")
print(f"  - {rel_output}/charities_stimuli.json")

Generated configuration files:
  - stimuli/main_studies/output_experiment_files/single-chat/single-chat.csv
  - stimuli/main_studies/output_experiment_files/influence-tasks/make-me-pay.csv
  - stimuli/main_studies/output_experiment_files/influence-tasks/change-my-view.csv
  - stimuli/main_studies/output_experiment_files/influence-tasks/make-me-return.csv

Generated stimuli files:
  - stimuli/main_studies/output_experiment_files/polchat_stimuli.json
  - stimuli/main_studies/output_experiment_files/emotchat_stimuli.json
  - stimuli/main_studies/output_experiment_files/moral_dilemmas_stimuli.json
  - stimuli/main_studies/output_experiment_files/charities_stimuli.json
